____
__Universidad Tecnológica Nacional, Buenos Aires__<br/>
__Ingeniería Industrial__<br/>
__Cátedra de Ciencia de Datos - Curso I5521 - Turno sabado mañana__<br/>
__Clase_02: EDA : Analisis Exploratorio de los Datos__<br/>
__Elaborado por: Nicolas Aguirre__
____

# Librerias

In [ ]:
#Importar paquetes de herramientas:
#Datos
import pandas as pd
import numpy as np
#Graficos 
import matplotlib.pyplot as plt
import seaborn as sns
#Otros 
import warnings
warnings.filterwarnings('ignore')

# Dataset

El dataset  que usaremos se encuentra en:
  
**https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data**

* Descargar e importarlo


* Verificar que se haya cargado bien el dataset


* Dimension del dataset

In [ ]:
df = pd.read_csv('clusterai_2020_clase02_dataset_ny_airbnb.csv',index_col="id")

In [ ]:
print(f'Shape: {np.shape(df)}')
df.head(3)

# Limpieza

* **Duplicados**


* **Cantidad** y **% de NaN** por columna


* Luego de quitarlos, devolver shape del df

In [ ]:
df.drop_duplicates(inplace=True,keep='first')

In [ ]:
cant_NaN = df.isnull().sum()
print(cant_NaN,'\r\n')

In [ ]:
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(6)

In [ ]:
#Limpiamos registros
df.dropna(how ='any', inplace = True)

In [ ]:
# Verificamos como quedo el data-set
print(f'Nos quedamos con un df de {df.shape[0]} filas x{df.shape[1]} columnas')

# Practica!

1. Crear columna $\log($precio$)$. Grafico de la distribucion de precio y de $\log($precio$)$


2. En base al que mejor se ajuste, comparar los precios segun los distintos *neighbourhood_group* (boxplot)


3. Mediana, Limite inferior y superior del *neighbourhood_group* con mayor y menor precio.


4. Crear una nueva columa llamada *Brand*, con 7 categorias de precios. 
        
    Bandas = [0, 50, 75, 100, 150, 200, 250]


5. Boxplot de *Brand* vs $\log($Reviews$)$


6. Scatter plot de Longitud y Latitud donde se distinga *neighbourhood_group*


7. Scatter plot de Longitud y Latitud donde se distingan las brands


8. Cantidad de barrios


9. Comparacion de la cantidad de hospedajes para los 10 barrios mas populares (barras)


10. Cantidad de hospedajes para los 10 barrios mas populares, segregados por brands (pivot_table)


## Log de precio y dist. estimada
Crear columna $\log($precio$)$. Grafico de la distribucion de precio y de $\log($precio$)$


In [ ]:
#Columna log precios
df['log_price'] = np.log(1+df['price'])

#Plot Precio
sns.histplot(df['price'],bins=50,color='b',kde=True,stat='count')
plt.xlabel("Price")
plt.title('Distribucion de Precio',size = 20)
plt.show()

#Plot Log-Precio
sns.histplot(np.log(1+df['price']),bins=50,color='b',kde=True,stat='count')
plt.xlabel("Price")
plt.show()

## Box plot *neighbourhood_group*

In [ ]:
#En base al que mejor se ajuste, comparar los precios segun los distintos "neighbourhood_group" (boxplot)
g = sns.catplot(data=df,
                x='neighbourhood_group',y='log_price', 
                kind="box", height = 5 ,aspect=3,palette = "muted")
g.despine(left=True)
g.set_xticklabels(rotation=90)
plt.xlabel("Barrio",fontsize=32)
plt.ylabel("Log_Precio",fontsize=32)
plt.title('Plot de log_price vs Barrio',size = 42)
plt.show()

## Mediana, Lim. Inferior y Sup.

In [ ]:
# Mediana, Limite inferior y superior del 'neighbourhood_group' con mayor y menor precio.

grup1 = 'Manhattan'
df_pivot = df[df['neighbourhood_group']==grup1]
plt.figure()
A = plt.boxplot(df_pivot['log_price'])
plt.title(grup1)
medians = [median.get_ydata() for median in A["medians"]]
#whiskers
whiskers = [whiskers.get_ydata() for whiskers in A["whiskers"]]
#print(f'Valores outliers:\r\n{outliers[0]}\r\n')
df2delete = pd.DataFrame({'Mediana': medians[0][0],
                            'Lim. Inferior': whiskers[0][0],
                            'Lim. Superior': whiskers[1][1]},index=[grup1])
display(df2delete)
grup2 = 'Bronx'
df_pivot= df[df['neighbourhood_group']==grup2]
plt.figure()
A = plt.boxplot(df_pivot['log_price'])
plt.title(grup2)
medians = [median.get_ydata() for median in A["medians"]]
#whiskers
whiskers = [whiskers.get_ydata() for whiskers in A["whiskers"]]
df2delete = pd.DataFrame({'Mediana': medians[0][0],
                            'Lim. Inferior': whiskers[0][0],
                            'Lim. Superior': whiskers[1][1]},index=[grup2])
display(df2delete)

#print(f'Valores outliers:\r\n{outliers[0]}\r\n')

## Brands

`
    Bandas = [0, 50, 75, 100, 150, 200, 250]
`

Podriamos hacerlo como la clase pasada ... pero veamos una manera usando **for loop** y **enumerate**. 

In [ ]:
# df.loc[(df['price'] > 0) & (df['price'] <= 50), 'Banda'] = 1
# df.loc[(df['price'] > 50) & (df['price'] <= 75), 'Banda']   = 2
# df.loc[(df['price'] > 75) & (df['price'] <= 100), 'Banda']   = 3
# df.loc[(df['price'] > 100) & (df['price'] <= 150), 'Banda']   = 4
# df.loc[(df['price'] > 150) & (df['price'] <= 200), 'Banda']   = 5
# df.loc[(df['price'] > 200) & (df['price'] <= 250), 'Banda']   = 6
# df.loc[(df['price'] > 250), 'Banda']  = 7

In [ ]:
#Definimos y creamos las bandas:
list_bandas = [0,50,75,100,150,200,250]

# Usaremos un 'for loop' con 'enumerate':
# El enumerate simplemente agrega al iterador una variable que "enumera" las iteraciones.
for i, i_banda in enumerate(list_bandas,start=1):
    if i < len(list_bandas):
        banda_i = list_bandas[i-1]
        banda_f = list_bandas[i]
        print(f"Enumerate --> i= {i}, banda_i= {banda_i}, banda_f= {banda_f}")
        df.loc[(df['price'] > banda_i) & (df['price'] <= banda_f), 'Banda']   = i
    else:
        banda_f = i_banda
        print(f"Enumerate --> i= {i}, banda_f= {banda_f}")
        df.loc[(df['price'] > banda_f), 'Banda']  = i

## Brand vs $\log($Reviews$)$

In [ ]:
#Boxplot de "Bandas" vs log(Reviews)
df['log_review'] = np.log(1+df['number_of_reviews'])

g = sns.catplot(data=df,
                x="Banda",y='log_review', 
                kind="box", height = 5,aspect=3 ,palette = "muted")
g.despine(left=True)
g.set_xticklabels(size=16,rotation=90)
g = g.set_ylabels("log_Reviews",size=32)
g = g.set_xlabels("Banda",size=22)
plt.title('Boxen plot Bandas VS Reviews',size = 32)
plt.show()

## Scatter Plot: *neighbourhood_group*

In [ ]:
plt.figure(figsize=(10,8))
sns.scatterplot(x=df.longitude,y=df.latitude,
                hue=df.neighbourhood_group,palette='muted')
plt.xlabel("Longitude",fontsize=20)
plt.xlabel("Latitude",fontsize=20)

plt.show()

## Scatter Plot: Brands

In [ ]:
# Podemos aplicar tambien algunas funciones de plot sobre el dataframe usando pd.plot( )
df.plot(kind='scatter', x='longitude', y='latitude', c='Banda',
        cmap=plt.get_cmap(), colorbar=True, alpha=0.5, figsize=(10,8))
plt.show()

## Qty de barrios

In [ ]:
q_barrios = len(df['neighbourhood'].unique())
print(f'En total hay {q_barrios} barrios\r\n')

## Hospedajes para los 10 barrios mas populares

In [ ]:
top_neighbourhood = df['neighbourhood'].value_counts().index[0:10]


plt.figure(figsize=(15,3))
g = sns.countplot(x='neighbourhood',data=df, palette = "muted",order=top_neighbourhood)
g.set_xticklabels(g.get_xticklabels(), rotation=90, ha="right")
plt.title('Cuenta por Barrio',size = 36)
plt.xlabel("Barrio",size = 24)
plt.ylabel("Count",size = 24)
plt.show()

## Cantidad de hospedajes para los 10 barrios mas populares, segregados por brands 

### Pivot Table

Veamos un poequeñp tutorial de [**Pivot Tables**](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pivot_table.html)

In [ ]:
#Generamos un datafame de ejemplo
df2delete = pd.DataFrame({"A": ["foo", "foo", "foo", "foo", "foo",
                         "bar", "bar", "bar", "bar"],
                   "B": ["one", "one", "one", "two", "two",
                         "one", "one", "two", "two"],
                   "C": ["small", "large", "large", "small",
                         "small", "large", "small", "small",
                         "large"],
                   "D": [1, 2, 2, 3, 3, 4, 5, 6, 7],
                   "E": [2, 4, 5, 5, 6, 6, 8, 9, 9]})
df2delete

In [ ]:
# Como se utiliza ...
table = pd.pivot_table(df2delete, # Datos
                       values=['D'], # Valores que van a aparecer dentro de la tabla
                       index=['A', 'B'], # Los indices que ve a tener
                    columns=['C'], # La columna que va a tener
                       aggfunc=np.sum) # Que se aplicara a los valores
table

In [ ]:
table = pd.pivot_table(df2delete,
                       # Multiples valores
                       values=['D', 'E'],
                       index=['A'],
                       # Tambien por columna
                       columns=['C'],
                       # Y con multiples funciones para cada uno de los valores
                       aggfunc={'D': np.mean,
                                'E': ['min', max, np.mean]})
table

In [ ]:
table = pd.pivot_table(
    # Datframe
    df[df['neighbourhood'].isin(top_neighbourhood)],
    # Filas
    index='neighbourhood',
    # Columnas
    columns = 'Banda',
    # Quiero ver los log_precios
    values='log_price' ,
    # si hay NaN, pongo un cero 
    fill_value=0,
    # Que funcion aplico?
    aggfunc = 'count')
table

# Consultas ?